In [1]:
import pandas as pd
import numpy as np
import json
import random

In [2]:
df_items = pd.read_csv('data/amazon_mobiles/items.csv')
df_reviews = pd.read_csv('data/amazon_mobiles/reviews.csv')

In [3]:
df_items.drop(['Unnamed: 0', 'url', 'reviewUrl', 'screen_size', 'memory_storage_capacity', 'originalPrice', 'totalRatings', 'about'], inplace=True, axis=1)
df_items.rename(columns={'title': 'name', 'rating_item': 'rating', 'asin': 'id', 'more': 'about'}, inplace=True)

In [4]:
df_items.drop_duplicates(subset=['name'], inplace=True)
df_items.dropna(subset=['name'], inplace=True)

In [5]:
df_items['price'] = [round(random.uniform(0.01, 300), ndigits=2) for i in range(df_items.shape[0])]
df_items['store'] = 'mobiles'
df_items['category'] = df_items.apply(lambda x: ["mobiles"], axis=1)
df_items['rating'] = [round(random.uniform(0.0, 5.0), ndigits=1) for i in range(df_items.shape[0])]

In [6]:
df_reviews.drop(['verified', 'helpfulVotes', 'country'], inplace=True, axis=1)

In [7]:
def getReviews(row, reviews_parsed):
    aux = {'author': row['name'], 'rating': row['rating'], 'date': row['date'], 'title': row['title'], 'body': row['body']}
    reviews_parsed.append(aux)

In [8]:
def mergeReviews(row):
    reviews = df_reviews.loc[df_reviews['asin'] == row['id']]
    reviews_parsed = []
    reviews.apply(getReviews, reviews_parsed=reviews_parsed, axis=1)
    return reviews_parsed

df_items['reviews'] = df_items.apply(mergeReviews, axis=1)

In [9]:
df_items['Id'] = ["products/mobiles/" + str(x) for x in range(1, len(df_items) + 1)]
df_items['stock'] = np.random.randint(0, 300, df_items.shape[0])
df_items.drop(columns='id', inplace=True)

In [10]:
df_items.to_csv('data/amazon_mobiles/products.csv', encoding='utf-8', index=False)

In [11]:
parsed = json.loads(df_items.to_json(orient="records", default_handler=str))

json_string = json.dumps(parsed, indent=4)
with open('data/output/amazon_mobiles.json', 'w') as outfile:
    outfile.write(json_string)